In [20]:
import pandas as pd
import awswrangler as wr
from application.modeling.constants import MY_SESSION, S3_CLIENT, FileStage

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

In [21]:
bucket_name = "zimnat-budgeting"
tenant_name = bucket_name
project_id = 1
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12

In [22]:
file_stage = FileStage.raw

parameters = wr.s3.read_parquet(
    f"s3://{bucket_name}/project_{project_id}/{file_stage.value}/parameters.parquet",
    boto3_session=MY_SESSION,
)
parameters

,Unnamed:_0,1,2,3,4,5,6,7,8,9,10,11,12
0,WORKING_CAPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NEW_TRADE_PAYABLES,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00
2,NEW_TRADE_RECEIVABLES,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00
3,RECEIPTS_FROM_TRADE_RECEIVABLES,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
4,PAYMENTS_TO_TRADE_PAYABLES,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
5,TRADE_RECEIVABLES_PROVISION_FOR_BAD_DEBTS,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,INVENTORIES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,INTERGROUP_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,OTHER_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,OTHER_PAYABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [23]:
from application.modeling import constants, helper


parameters = helper.read_parameters_file(
    tenant_name=tenant_name,
    project_id=project_id,
    boto3_session=constants.MY_SESSION,
    valuation_date=VALUATION_DATE,
)

In [26]:
helper.change_period_index_to_strftime(
    parameters.loc['DIVIDEND_PAID']
)

Jan-2023     0.00
Feb-2023     0.00
Mar-2023     0.00
Apr-2023     0.00
May-2023     0.00
Jun-2023     0.00
Jul-2023     0.00
Aug-2023     0.00
Sep-2023     0.00
Oct-2023     0.00
Nov-2023     0.00
Dec-2023   100.00
Name: DIVIDEND_PAID, dtype: float64

In [25]:
parameters.loc['DIVIDEND_PAID']

2023-01     0.00
2023-02     0.00
2023-03     0.00
2023-04     0.00
2023-05     0.00
2023-06     0.00
2023-07     0.00
2023-08     0.00
2023-09     0.00
2023-10     0.00
2023-11     0.00
2023-12   100.00
Name: DIVIDEND_PAID, dtype: float64